In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

mask_on = False

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
nose_cascade = cv2.CascadeClassifier('Nariz_Hidung.xml')

cap = cv2.VideoCapture(0)
ret,frame1 = cap.read()

prev_img = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255


while True:
    _, frame = cap.read()
    nextImg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_img,nextImg,None,0.5,3,15,3,5,1.2,0)
    
    mag,ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees = True)
    
    hsv_mask[:,:,0] = ang/2
    
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    
    cv2.imshow('frame',bgr)
    k = cv2.waitKey(30) & 0xFF
    
    if k == 27:
        break
        
        prev_img = nextImg

#deteksi wajah

    face = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in face:
        if mask_on:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 3)
            cv2.putText (frame,'Mask on', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2,(0, 255, 0), 5)
        else:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText (frame,'Mask off', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 2,(0, 255, 0), 5)
            

        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
       
       #Deteksi mata
        eye = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eye:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            cv2.putText (frame,'eye', (x + ex,y +ey), 1, 2,(0, 255, 0), 2)
            

            #Deteksi hidung 
            nose = nose_cascade.detectMultiScale(gray, 1.18, 35)
            for (sx,sy,sw,sh) in nose:
                cv2.rectangle(frame, (sx, sy), (sx+sw, sy+sh), (255, 0, 0) ,2)
                cv2.putText (frame,'nose', (x + sx,y +sy), 1, 3,(255, 0, 0), 2)

            if len(nose)>0:
                mask_on = False
            else:
                mask_on = True

    cv2.putText (frame,'jumlah wajah : ' + str(len(face)), (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 255, 0), 2)
    cv2.imshow('Face', frame)

    
if cv2.waitKey(30) & 0xff == ord('q'):
    break 
cap.release()
cv2.destroyAllWindows()


: 

: 